In [ ]:
import numpy as np
import pandas as pd
import json
import gzip
import ast
import matplotlib.pyplot as plt

from urllib.request import urlopen
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import defaultdict
from random import sample 
from scipy.sparse import lil_matrix
from collections import Counter

In [ ]:
def readJSON(path):
    for l in gzip.open(path, 'rt'):
        d = eval(l)
        u = d['userID']
        try:
            g = d['gameID']
        except Exception as e:
            g = None
        yield u,g,d

gameCount = Counter()

for user,game,_ in readJSON("train.json.gz"):
    gameCount[game] += 1


In [ ]:
return1 = [x[0] for x in gameCount.most_common(200)]

In [ ]:
def parseDataFromFile(fname):
    for l in open(fname):
    #yield eval(l)
        yield ast.literal_eval(l)

data = list(parseDataFromFile("train.json"))
train_data = data[:165000]
test_data = data[165000:]

user_dict = defaultdict(list) # games played by user
game_dict = defaultdict(list) # users who played game
game_set = set()

for row in train_data:
    user_dict[row['userID']].append(row['gameID'])
    game_dict[row['gameID']].append(row['userID'])
    game_set.add(row['gameID'])
    
positives = [[x['userID'], x['gameID'], 1] for x in test_data]

negatives = []

for row in test_data:
    not_played = [x for x in game_set if x not in user_dict[row['userID']]]
    negatives.append([row['userID'], sample(not_played, 1)[0], 0])

X_val = positives + negatives
y_val = [x[2] for x in X_val]

In [ ]:
def Jaccard(s1, s2):
    s1 = set(s1)
    s2 = set(s2)
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [ ]:
def predictPlayed(user,game):
    similarities = []
    for g in user_dict[user]:
        if g == game: continue
        similarities.append(Jaccard(game_dict[game],game_dict[g]))
    if len(similarities) == 0:
        return 0
    if (np.mean(similarities) > .009):
        return 1
    else:
        # User hasn't rated any similar items
        return 0

In [ ]:
pred_list = []
for l in X_val:
    u,g = l[0], l[1]
    
    pred_list.append(predictPlayed(u,g))


In [ ]:
TP_ = np.logical_and(pred_list, y_val)
FP_ = np.logical_and(pred_list, np.logical_not(y_val))
TN_ = np.logical_and(np.logical_not(pred_list), np.logical_not(y_val))
FN_ = np.logical_and(np.logical_not(pred_list), y_val)
TP = sum(TP_)
FP = sum(FP_)
TN = sum(TN_)
FN = sum(FN_)
print(TP)
print(FP)
print(TN)
print(FN)

In [ ]:
accuracy_score(pred_list, y_val)

In [ ]:
### kaggle stuff below
X_test = []

f = open("pairs_Played.txt", "r")
f.readline()
for x in f:
    split = x.strip('\n').split('-')
    X_test.append(split)

In [ ]:
predictions = open("predictions_Played.txt", 'w')
predictions.write('userID-gameID,prediction\n')

for i in range(len(X_test)):
    u,g = X_test[i][0], X_test[i][1]

    if pred_kaggle[i] == 1:
        predictions.write(u + '-' + g + "," + str(predictPlayed(u,g)) + "\n")

    else:
        predictions.write(u + '-' + g + "," + str(predictPlayed(u,g)) + "\n")


predictions.close()